# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pickle
import pandas as pd
pd.set_option('display.max_columns', 200)

from card import Card
from board import Board
from player import Player
from rule import Rule
import optimal_strategy as opt

import multiprocessing
import time
from multiprocessing.pool import ThreadPool
import threading
from tqdm import notebook
notebook.tqdm.pandas()

from euchre_lib import *
from search_lib import *
import data_formatting as dtf

In [3]:
notebook.tqdm.get_lock().locks = []

# Create dataset

In [211]:
os.system('rm -r RLdataset/*')

0

In [212]:
mx_thresh = 72; N_CPUS = 8
thresholds = [mx_thresh]*N_CPUS
n_hands = int(1e4)
n_epochs = 100
# get 8 million = 8e6 = 8 CPUS x 1e4 hands x 1e2 epochs
# estimate it takes 35 minutes, given 500 hands/sec
# actually more like 90-100 minutes, eh
if not os.path.exists(os.path.join(os.getcwd(), 'RLdataset')):
    os.mkdir(os.path.join(os.getcwd(), 'RLdataset'))

In [213]:
parallel_search_wrapper(thresholds, MAX_CPUS=N_CPUS, n_epochs=n_epochs, n_hands=n_hands,\
                        opp_thresh=mx_thresh, title='Making dataset', folder='RLdataset')

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Done!
This took 4722.23 seconds
This took 78.70 minutes


In [ ]:
#def parallel_search_wrapper(thresholds, MAX_CPUS=8, n_epochs=10, n_hands=100, opp_thresh=None,
#                            title=None, ROOT_DIR=os.getcwd()+'/', folder=None):

# Format Data

In [6]:
infolder = os.path.join(os.getcwd(), 'RLdataset')
#infolder = os.path.join(os.getcwd(), 'thresholds', 'thresh70')
data = read_all_hands(infolder)
data.shape

  0%|          | 0/100 [00:00<?, ?it/s]

(1000000, 122)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 122 entries, p0c1 to p3trueid
dtypes: int64(101), object(21)
memory usage: 930.8+ MB


In [8]:
data.head()

,p0c1,p0c1isD,p0c1isH,p0c1isS,p0c2,p0c2isD,p0c2isH,p0c2isS,p0c3,p0c3isD,p0c3isH,p0c3isS,p0c4,p0c4isD,p0c4isH,p0c4isS,p0c5,p0c5isD,p0c5isH,p0c5isS,p1c1,p1c1isD,p1c1isH,p1c1isS,p1c2,p1c2isD,p1c2isH,p1c2isS,p1c3,p1c3isD,p1c3isH,p1c3isS,p1c4,p1c4isD,p1c4isH,p1c4isS,p1c5,p1c5isD,p1c5isH,p1c5isS,p2c1,p2c1isD,p2c1isH,p2c1isS,p2c2,p2c2isD,p2c2isH,p2c2isS,p2c3,p2c3isD,p2c3isH,p2c3isS,p2c4,p2c4isD,p2c4isH,p2c4isS,p2c5,p2c5isD,p2c5isH,p2c5isS,p3c1,p3c1isD,p3c1isH,p3c1isS,p3c2,p3c2isD,p3c2isH,p3c2isS,p3c3,p3c3isD,p3c3isH,p3c3isS,p3c4,p3c4isD,p3c4isH,p3c4isS,p3c5,p3c5isD,p3c5isH,p3c5isS,TC_power,TC_isD,TC_isH,TC_isS,trump_isD,trump_isH,trump_isS,caller,round,alone,winner1,winner2,winner3,winner4,winner5,points02,points13,played1,played2,played3,played4,played5,played6,played7,played8,played9,played10,played11,played12,played13,played14,played15,played16,played17,played18,played19,played20,result,p0trueid,p1trueid,p2trueid,p3trueid
0,20,0,1,0,12,0,1,0,10,0,0,0,1,0,0,0,2,0,0,1,35,0,1,0,31,1,0,0,4,0,0,0,3,0,0,0,1,0,0,1,10,1,0,0,4,1,0,0,1,1,0,0,5,0,0,1,4,0,0,1,30,0,1,0,5,1,0,0,2,1,0,0,10,0,0,1,3,0,0,1,15,0,1,0,0,1,0,0,1,0,0,0,3,0,3,1,0,JH0,9D1,AH2,9H3,JD0,QD1,TD2,TH3,QC0,QS1,JS2,AC3,9C3,JC0,KS1,KD2,9S0,AD1,AS2,QH3,Single,3,0,1,2
1,30,0,0,0,15,0,0,0,4,0,1,0,2,0,1,0,10,0,0,1,20,0,0,0,5,1,0,0,2,1,0,0,5,0,0,1,4,0,0,1,31,0,0,1,10,1,0,0,3,1,0,0,5,0,1,0,2,0,0,1,35,0,0,0,12,0,0,0,1,1,0,0,3,0,1,0,1,0,1,0,25,0,0,0,0,0,0,1,1,0,0,3,3,3,3,0,1,JC0,TC1,QC2,KC3,JH0,QH1,TD2,KH3,AD3,9D0,TH1,KD2,JS3,9C0,AC1,QS2,JD3,9H0,AS1,KS2,Single,0,1,2,3
2,3,0,0,0,5,0,1,0,10,0,0,1,5,0,0,1,1,0,0,1,30,1,0,0,25,1,0,0,20,1,0,0,12,1,0,0,2,0,1,0,31,0,1,0,10,0,0,0,2,0,0,0,4,0,1,0,2,0,0,1,35,1,0,0,4,0,0,0,1,0,0,0,1,0,1,0,3,0,0,1,15,1,0,0,1,0,0,0,1,0,2,0,0,0,0,2,0,AD0,JH1,JD2,TD3,QC2,JC3,9D0,TC1,KD0,TS1,9C2,KH3,QD0,QH1,9H2,KS3,TH0,AC1,JS2,AS3,Sweep,1,2,3,0
3,35,0,0,0,30,0,0,0,12,0,0,0,10,1,0,0,1,0,1,0,5,1,0,0,4,1,0,0,10,0,1,0,4,0,1,0,10,0,0,1,15,0,0,0,2,1,0,0,3,0,1,0,5,0,0,1,4,0,0,1,31,0,0,1,3,1,0,0,5,0,1,0,2,0,1,0,2,0,0,1,25,0,0,0,0,0,0,1,1,0,2,1,1,1,1,0,1,KH0,9H1,AH2,JH3,AS2,QS3,TS0,9C1,JC1,QD2,TC3,JS0,AD1,KD2,KS3,JD0,AC1,QH2,KC3,TH0,Single,2,3,0,1
4,10,0,0,0,4,0,0,0,10,1,0,0,2,0,1,0,1,0,1,0,5,1,0,0,4,1,0,0,1,1,0,0,5,0,1,0,4,0,1,0,31,0,0,0,25,0,0,1,2,0,0,0,1,0,0,0,3,0,1,0,30,0,0,1,20,0,0,1,12,0,0,1,5,0,0,0,2,1,0,0,35,0,0,1,0,0,1,1,1,0,1,2,3,3,2,0,1,KD0,KS1,TD2,AD3,JH1,9S2,TH3,QH0,KC2,AC3,9D0,9C1,JS3,QD0,JC1,QS2,QC3,KH0,TC1,AS2,Single,3,0,1,2


In [9]:
p0hand = ['JH', 'JD', 'QC', 'JC', '9S']
p1hand = ['AD', 'QD', '9D', 'KS', 'QS']
p2hand = ['AH', 'KD', 'TD', 'AS', 'JS']
p3hand = ['QH', '9H', 'AC', '9C', 'TS'] + ['TH']

### Check that the data formatting functions work properly

In [159]:
# this seems to be working just fine
group1_info = dtf.get_group1_info(data.iloc[1], 0, 0)
for arr in [group1_info[:4], group1_info[4:8], group1_info[8], group1_info[9], group1_info[10:14], group1_info[14:]]:
    print(arr)
z="""
    #######
    DATA GROUP 1: Calling trump
    #######
    (4)  1.) Who dealt (position relative to "me")
    (4)  2.) Who called trump (position relative to "me")
    (1)  3.) Which round was trump called in
    (1)  4.) Going alone?
    (4)  5.) Which suit is trump (not sure if this one needs to be here)
    (10) 6.) What is the turn card
    Total: 24 columns
    """

[0 0 1 0]
[1 0 0 0]
1
0
[1 0 0 0]
[0 0 0 0 1 0 1 0 0 0]


In [149]:
# this seems to be working just fine
tnum = 4
group2_info = dtf.get_group2_info(data.iloc[0], tnum, 3)
for arr in [group2_info[:4], group2_info[4:8], group2_info[8:19], group2_info[19:30], group2_info[30:35], group2_info[35:]]:
    print(arr)
print('Led card:', data.iloc[0]['played'+str(4*tnum+1)])

z="""
    #######
    DATA GROUP 2: Other misc. information
    #######
    (4)  1.) Who is leading right now
    (4)  2.) Who is winning right now
    (11) 3.) What card was led (is it trump)
    (11) 4.) What card is winning (is it trump)
    (5)  5.) Which team won each trick so far (+1 for "me", 0 for no one (yet), -1 for opponents)
    (12) 6.) Any players confirmed short in any suits
    Total: 47 columns
    """

[1 0 0 0]
[0 0 1 0]
[1 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 1 0]
[-1 -1  1 -1  0]
[0 0 0 0 1 0 1 1 1 0 0 0]
Led card: 9S0


In [82]:
# this seems to be working just fine
num_cols = 13
group3_info = dtf.get_group3_info(data.iloc[0], 0, 0)
for i in range(num_cols):
    print(i, ': ', sum(group3_info[i::num_cols]))

print()
print(p3hand)
print()
cards = [c+s for s in list('CDHS') for c in list('9TJQKA')]
for i in range(24):
    print(cards[i], ': ', group3_info[(num_cols*i):(num_cols*(i+1))])
z="""
    #######
    DATA GROUP 3: All card locations (constant order: 9C, 10C, ..., (D), (H), ..., KS, AS)
    #######
    For each card (24):
    (4)  1.) Confirmed in anyone's hand (my hand + ordered up turn card?)
    (4)  2.) Played in a previous trick by someone (maybe later expand this to which prev trick?)
    (3)  3.) Played in CURRENT trick by someone
    (1)  4.) Confirmed buried
    (1)  5.) Is trump
    Total: 312 columns
    """

0 :  0
1 :  0
2 :  1
3 :  5
4 :  0
5 :  0
6 :  0
7 :  0
8 :  0
9 :  0
10 :  0
11 :  0
12 :  7

['QH', '9H', 'AC', '9C', 'TS', 'TH']

9C :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
TC :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
JC :  [0 0 0 1 0 0 0 0 0 0 0 0 0]
QC :  [0 0 0 1 0 0 0 0 0 0 0 0 0]
KC :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
AC :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
9D :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
TD :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
JD :  [0 0 0 1 0 0 0 0 0 0 0 0 1]
QD :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
KD :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
AD :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
9H :  [0 0 0 0 0 0 0 0 0 0 0 0 1]
TH :  [0 0 1 0 0 0 0 0 0 0 0 0 1]
JH :  [0 0 0 1 0 0 0 0 0 0 0 0 1]
QH :  [0 0 0 0 0 0 0 0 0 0 0 0 1]
KH :  [0 0 0 0 0 0 0 0 0 0 0 0 1]
AH :  [0 0 0 0 0 0 0 0 0 0 0 0 1]
9S :  [0 0 0 1 0 0 0 0 0 0 0 0 0]
TS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
JS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
QS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
KS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
AS :  [0 0 0 0 0 0 0 0 0 0 0 0 0]


In [44]:
# this seems to be working just fine
group4_info = dtf.get_group4_info(data.iloc[0], 0, 0)
print(group4_info, '\n\n', p0hand, '\n\n', (np.argwhere(np.array(group4_info)==1).reshape(-1)+1)%11)

[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 

 ['JH', 'JD', 'QC', 'JC', '9S'] 

 [ 4  7  3  9  0  3  8  0  3  7  1 10]


### Format the first 10k hands csv

In [221]:
folder = os.path.join(os.getcwd(), 'RLdataset')
first_group = pd.concat([pd.read_csv(os.path.join(folder, '10000_hands_'+str(i).zfill(3)+'.csv'))\
                         for i in notebook.tqdm(range(2))]).reset_index(drop=True)
first_group.head()

  0%|          | 0/2 [00:00<?, ?it/s]

,p0c1,p0c1isD,p0c1isH,p0c1isS,p0c2,p0c2isD,p0c2isH,p0c2isS,p0c3,p0c3isD,p0c3isH,p0c3isS,p0c4,p0c4isD,p0c4isH,p0c4isS,p0c5,p0c5isD,p0c5isH,p0c5isS,p1c1,p1c1isD,p1c1isH,p1c1isS,p1c2,p1c2isD,p1c2isH,p1c2isS,p1c3,p1c3isD,p1c3isH,p1c3isS,p1c4,p1c4isD,p1c4isH,p1c4isS,p1c5,p1c5isD,p1c5isH,p1c5isS,p2c1,p2c1isD,p2c1isH,p2c1isS,p2c2,p2c2isD,p2c2isH,p2c2isS,p2c3,p2c3isD,p2c3isH,p2c3isS,p2c4,p2c4isD,p2c4isH,p2c4isS,p2c5,p2c5isD,p2c5isH,p2c5isS,p3c1,p3c1isD,p3c1isH,p3c1isS,p3c2,p3c2isD,p3c2isH,p3c2isS,p3c3,p3c3isD,p3c3isH,p3c3isS,p3c4,p3c4isD,p3c4isH,p3c4isS,p3c5,p3c5isD,p3c5isH,p3c5isS,TC_power,TC_isD,TC_isH,TC_isS,trump_isD,trump_isH,trump_isS,caller,round,alone,winner1,winner2,winner3,winner4,winner5,points02,points13,played1,played2,played3,played4,played5,played6,played7,played8,played9,played10,played11,played12,played13,played14,played15,played16,played17,played18,played19,played20,result,p0trueid,p1trueid,p2trueid,p3trueid
0,12,1,0,0,4,0,1,0,2,0,1,0,1,0,1,0,1,0,0,1,25,1,0,0,4,0,0,0,3,0,0,0,5,0,1,0,5,0,0,1,10,0,0,0,5,0,0,0,10,0,0,1,3,0,0,1,2,0,0,1,31,0,1,0,15,1,0,0,2,0,0,0,10,0,1,0,4,0,0,1,30,1,0,0,1,0,0,3,1,0,3,3,2,3,3,0,1,QH0,KH1,TS2,AH3,JH3,9D0,KD1,JS2,QS3,9S0,KS1,AS2,AC2,TD3,9H0,JC1,AD3,TH0,QC1,KC2,Single,3,0,1,2
1,35,0,0,1,20,0,0,1,10,0,0,0,4,0,0,0,10,1,0,0,30,0,0,1,3,1,0,0,10,0,1,0,5,0,1,0,3,0,1,0,31,0,0,0,15,0,0,1,4,1,0,0,2,1,0,0,1,0,1,0,25,0,0,1,1,0,0,0,5,1,0,0,1,1,0,0,4,0,1,0,12,0,0,1,0,0,1,0,1,0,2,0,0,3,0,1,0,AC0,AS1,JC2,QH3,QD2,KD3,AD0,JD1,JS0,JH1,TS2,9S3,QC0,KH1,9H2,KS3,9D3,QS0,AH1,TD2,Single,0,1,2,3
2,1,0,0,0,5,1,0,0,4,1,0,0,10,0,1,0,4,0,1,0,25,0,0,1,2,0,0,0,3,1,0,0,1,1,0,0,3,0,1,0,10,0,0,0,5,0,0,0,4,0,0,0,5,0,1,0,1,0,1,0,35,0,0,1,31,0,0,0,15,0,0,1,12,0,0,1,10,1,0,0,30,0,0,1,0,0,1,3,1,1,3,3,3,3,3,0,4,AH0,--1,9H2,9S3,JS3,9C0,--1,QC2,JC3,QD0,--1,KC2,AS3,QH0,--1,KH2,TS3,KD0,--1,AC2,Loner,1,2,3,0
3,2,0,0,1,30,0,0,0,20,0,0,0,15,0,0,0,3,0,1,0,4,0,0,1,12,0,0,0,2,1,0,0,1,1,0,0,1,0,1,0,35,0,0,0,25,0,0,0,4,1,0,0,3,1,0,0,10,0,1,0,5,0,0,1,1,0,0,1,10,1,0,0,5,0,1,0,2,0,1,0,35,0,0,1,0,0,0,0,2,0,2,2,0,0,3,1,0,JH0,9H1,AH2,TH3,JC2,9S3,TC0,9C1,JD2,AD3,QC0,9D1,AC0,TD1,KC2,KH3,TS0,QS1,QD2,KS3,Single,2,3,0,1
4,1,0,0,0,5,1,0,0,4,0,1,0,3,0,1,0,2,0,1,0,31,0,0,0,10,1,0,0,4,1,0,0,2,1,0,0,5,0,1,0,30,0,0,1,15,0,0,1,5,0,0,0,4,0,0,0,10,0,1,0,25,0,0,1,12,0,0,1,10,0,0,0,1,1,0,0,1,0,1,0,35,0,0,1,0,0,1,2,1,0,3,3,3,2,3,0,2,KD0,AD1,TS2,KS3,AC3,9C0,KH1,QC2,JS3,TH0,JC1,AS2,9H3,QH0,TD1,AH2,KC2,9S3,JH0,QD1,EUCHRE,3,0,1,2


In [222]:
first_formatted = dtf.format_data(first_group)

  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Do the rest of the formatting on a HPC (see `format_data.sh`)

In [5]:
expected_count_comma = 122
def count_odd_lines(infile):
    with open(infile, 'r') as f:
        lines = f.read()
    
    weird_lines = [[i, len(line.split(','))] for i, line in enumerate(lines.split('\n')) if len(line.split(',')) != expected_count_comma]
    
    return weird_lines

In [321]:
for i in range(500, 797):
    file = 'RLdataset/10000_hands_' + str(i) + '.csv'
    weird_lines = count_odd_lines(file)
    if len(weird_lines) > 1:
        print(file+':', len(weird_lines), weird_lines[:5])

RLdataset/10000_hands_500.csv: 64 [[1340, 243], [1815, 244], [1870, 244], [1897, 247], [1924, 247]]
RLdataset/10000_hands_611.csv: 5 [[306, 125], [334, 123], [361, 243], [10000, 13], [10001, 1]]
RLdataset/10000_hands_666.csv: 52 [[698, 154], [725, 255], [752, 228], [780, 145], [807, 250]]


In [323]:
for i in range(500, 797):
    file = 'RLdataset/10000_hands_' + str(i) + '.csv'
    weird_lines = count_odd_lines(file)
    if len(weird_lines) > 1:
        print('Re-doing file %s' %file)
        with open(file, 'r') as infile:
            dirty = infile.read()
        
        clean = '\n'.join([line for line in dirty.split('\n') if len(line.split(',')) == expected_count_comma])
        with open(file, 'w') as outfile:
            outfile.write(clean)
        
        weird_lines = count_odd_lines(file)
        if len(weird_lines) > 1:
            print('ERROR IN CLEANING FILE %s' %file)

Re-doing file RLdataset/10000_hands_500.csv
Re-doing file RLdataset/10000_hands_611.csv
Re-doing file RLdataset/10000_hands_666.csv


# Define model fn

In [9]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
print(tf.__version__)
import model_fns as mfn

1.15.3


In [86]:
model = mfn.make_model()
#model.summary() # about 7.4 mil trainable parameters

In [87]:
%%time
TRAIN, VALID, TEST = mfn.train_valid_test()

CPU times: user 5.68 s, sys: 218 ms, total: 5.9 s
Wall time: 5.9 s


In [88]:
%%time
for data, target in TRAIN.take(1):
    print(data.shape, target.shape)
    break

(10, 438) (10, 24)
CPU times: user 9.03 s, sys: 257 ms, total: 9.29 s
Wall time: 9.27 s


In [ ]:
history = model.fit(
    TRAIN,
    epochs=10,
    validation_data=VALID,
    steps_per_epoch=int(1e5),
    validation_steps=int(1e4),
    callbacks = [keras.callbacks.ModelCheckpoint('models/model1_original.h5')]
)

Train for 100000 steps, validate for 10000 steps
Epoch 1/10
100000/100000 [==============================] - 632s 6ms/step - loss: 0.3754 - categorical_accuracy: 0.8798 - precision_4: 0.9057 - recall_4: 0.8591 - auc_4: 0.9940 - val_loss: 0.3151 - val_categorical_accuracy: 0.8914 - val_precision_4: 0.9086 - val_recall_4: 0.8719 - val_auc_4: 0.9967- precis
Epoch 2/10
100000/100000 [==============================] - 625s 6ms/step - loss: 0.1461 - categorical_accuracy: 0.9511 - precision_4: 0.9571 - recall_4: 0.9461 - auc_4: 0.9983 - val_loss: 0.2193 - val_categorical_accuracy: 0.9275 - val_precision_4: 0.9355 - val_recall_4: 0.9177 - val_auc_4: 0.9978
Epoch 3/10
100000/100000 [==============================] - 638s 6ms/step - loss: 0.1057 - categorical_accuracy: 0.9647 - precision_4: 0.9684 - recall_4: 0.9615 - auc_4: 0.9988 - val_loss: 0.1721 - val_categorical_accuracy: 0.9433 - val_precision_4: 0.9484 - val_recall_4: 0.9376 - val_auc_4: 0.9981
Epoch 4/10
100000/100000 [=================

----------

How big actually are the train / valid datasets? They have ~400 columns each, and there's about $800 \times 10^4$ hands, giving roughly 160 million cards played. Then, 70% of this (about 110 million hands) goes into train, 20% of this goes into validation (about 32 million hands), and the final 10% (about 16 million hands) goes into test.

----------

In [13]:
model = make_model(dense_layers=[x*dtf.COLS_TOTAL for x in [2, 1, 0.5, 0.25]], drop_prob=0.1, reg=0.01)
#model.summary() # about 900k trainable parameters

In [18]:
# expect this to take like 1000 minutes = 16 hours
history2 = model.fit(
    TRAIN,
    epochs=100,
    validation_data=VALID,
    steps_per_epoch=int(1e5),  # 1e5 steps x 100 epochs x 10 per batch = 1e8 points = 100 million
    validation_steps=int(3e4), # 3e4 steps x 100 epochs x 10 per batch = 3e7 points =  30 million
    callbacks = [keras.callbacks.ModelCheckpoint('models/model2_slimmed.h5')]
)

Train for 100000 steps, validate for 30000 steps
Epoch 1/100
100000/100000 [==============================] - 542s 5ms/step - loss: 2.1905 - categorical_accuracy: 0.7527 - precision_5: 0.8203 - recall_5: 0.6851 - auc_5: 0.9730 - val_loss: 2.3771 - val_categorical_accuracy: 0.5976 - val_precision_5: 0.6394 - val_recall_5: 0.5481 - val_auc_5: 0.9407
Epoch 2/100
100000/100000 [==============================] - 535s 5ms/step - loss: 1.7358 - categorical_accuracy: 0.7754 - precision_5: 0.8404 - recall_5: 0.7160 - auc_5: 0.9745 - val_loss: 2.4974 - val_categorical_accuracy: 0.5764 - val_precision_5: 0.6217 - val_recall_5: 0.5264 - val_auc_5: 0.9235
Epoch 3/100
100000/100000 [==============================] - 540s 5ms/step - loss: 1.6737 - categorical_accuracy: 0.7791 - precision_5: 0.8449 - recall_5: 0.7212 - auc_5: 0.9745 - val_loss: 2.2888 - val_categorical_accuracy: 0.5906 - val_precision_5: 0.6370 - val_recall_5: 0.5367 - val_auc_5: 0.9410
Epoch 4/100
100000/100000 [=====================

KeyboardInterrupt: 

This doesn't seem to be getting much better over time. Try an intermediately complex model?

In [24]:
model = make_model(dense_layers=[x*dtf.COLS_TOTAL for x in [4, 2, 1, 0.5, 0.25]], drop_prob=0.1, reg=0.01)
#model.summary() # about 2.8 mil trainable parameters

In [26]:
# expect this to take like 1000 minutes = 16 hours
history3 = model.fit(
    TRAIN,
    epochs=100,
    validation_data=VALID,
    steps_per_epoch=int(1e5),  # 1e5 steps x 100 epochs x 10 per batch = 1e8 points = 100 million
    validation_steps=int(3e4), # 3e4 steps x 100 epochs x 10 per batch = 3e7 points =  30 million
    callbacks = [keras.callbacks.ModelCheckpoint('models/model3_intermediateSize.h5')]
)

Train for 100000 steps, validate for 30000 steps
Epoch 1/100
100000/100000 [==============================] - 604s 6ms/step - loss: 2.6679 - categorical_accuracy: 0.7306 - precision_11: 0.8104 - recall_11: 0.6506 - auc_11: 0.9638 - val_loss: 2.8032 - val_categorical_accuracy: 0.5694 - val_precision_11: 0.6215 - val_recall_11: 0.5050 - val_auc_11: 0.9161
Epoch 2/100
100000/100000 [==============================] - 600s 6ms/step - loss: 1.9980 - categorical_accuracy: 0.7618 - precision_11: 0.8356 - recall_11: 0.6944 - auc_11: 0.9665 - val_loss: 2.9937 - val_categorical_accuracy: 0.5419 - val_precision_11: 0.5863 - val_recall_11: 0.4866 - val_auc_11: 0.8906
Epoch 3/100
100000/100000 [==============================] - 603s 6ms/step - loss: 1.9159 - categorical_accuracy: 0.7658 - precision_11: 0.8409 - recall_11: 0.6995 - auc_11: 0.9665 - val_loss: 2.5687 - val_categorical_accuracy: 0.5929 - val_precision_11: 0.6382 - val_recall_11: 0.5422 - val_auc_11: 0.9233
Epoch 4/100
100000/100000 [===

In [65]:
for data, target in TRAIN.take(5):
    data, target = data.numpy(), target.numpy()
    #data, target = data[0], target[0]
    #print(data)
    print(data.shape, target.shape)

(10, 438) (10, 24)
(10, 438) (10, 24)
(10, 438) (10, 24)
(10, 438) (10, 24)
(10, 438) (10, 24)


In [64]:
remove_singular_plays(data, target)

Singular fraction:             0.80 (8 / 10)
Singular follow suit fraction: 0.50 (4 / 8)
Singular final trick fraction: 0.50 (4 / 8)


(array([[1, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 1, 0]], dtype=int8),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0]], dtype=int8))

In [63]:
def remove_singular_plays(data, target, keep_prob=0.1):
    # find ixs of single-option plays
    # assume inputs are numpy arrays
    singular_ixs = set()
    if len(data.shape) == 1: data = np.array([data])
    if len(target.shape) == 1: target = np.array([target])
        
    count_finaltrick, count_followsuit = 0,0
    for i in range(len(data)):
        row = data[i]
        COLS_COUNTS = [0, dtf.COLS_GROUP1, dtf.COLS_GROUP2, dtf.COLS_GROUP3, dtf.COLS_GROUP4]
        group1, group2, group3, group4 = [row[sum(COLS_COUNTS[:(i+1)]) : sum(COLS_COUNTS[:(i+2)])  ] for i in range(len(COLS_COUNTS)-1)]
        
        # self maps to 3 -- (0, 1, 2, 3)
        count_cards_remaining = sum(group3[3::int(dtf.COLS_GROUP3/24)])
        # alternate method -- use group4
        #count_cards_remaining = int(sum(  [sum(group4[11*i:11*(i+1)][:-1]) for i in range(5)]  )/2)
        if count_cards_remaining == 1:
            count_finaltrick += 1
            singular_ixs.add(i)
            continue
        
        led_suit = 0*group2[4+4+6 + 0] + 1*group2[4+4+6 + 1] + 2*group2[4+4+6 + 2] + 3*group2[4+4+6 + 3]
        led_trump = group2[4+4+7 + 4]
        
        if led_trump:
            trumps_in_hand = sum(group4[10::11])
            if trumps_in_hand == 1:
                count_followsuit += 1
                singular_ixs.add(i)
                continue
        else:
            followsuits_in_hand = sum([c[6+led_suit] and not c[-1] for c in [group4[11*i:11*(i+1)] for i in range(5)]])
            if followsuits_in_hand == 1:
                count_followsuit += 1
                singular_ixs.add(i)
                continue
                
    print('Singular fraction:             %.2f (%i / %i)' %(len(singular_ixs)/len(data), len(singular_ixs), len(data)))
    print('Singular follow suit fraction: %.2f (%i / %i)' %(count_followsuit/len(singular_ixs), count_followsuit, len(singular_ixs)))
    print('Singular final trick fraction: %.2f (%i / %i)' %(count_finaltrick/len(singular_ixs), count_finaltrick, len(singular_ixs)))
    #return singular_ixs
    keep_ixs = [i for i in singular_ixs if np.random.random() < keep_prob]
    keep_ixs = np.array(keep_ixs + [i for i in range(len(data)) if i not in singular_ixs])
    
    return data[keep_ixs], target[keep_ixs]